In [48]:
#@markdown ####Fetch
%cd

!git clone -b dev https://github.com/jzcurious/cuda-sample1.git
%cd cuda-sample1

project_dir = !pwd
project_dir = project_dir[0]


/root
Cloning into 'cuda-sample1'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 33 (delta 1), reused 33 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 13.59 KiB | 13.59 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/root/cuda-sample1


####Debug build (host)

In [49]:
%cd $project_dir
!mkdir -p build
%cd build
!cmake --fresh -DTESTING=1 -DCMAKE_BUILD_TYPE=DebugHost ..
!cmake --build . -j4
%cd tests

/root/cuda-sample1
/root/cuda-sample1/build
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Host debugging mode is enabled
-- Testing is enabled
Cloning into 'googletest-src'...
HEAD is now at b514bdc8 Update version strings to 1.15.2 (#4583)
-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Performing T

In [50]:
!ASAN_OPTIONS="protect_shadow_gap=0" ./cuda-sample1-test --gtest_brief=1

Running main() from /root/cuda-sample1/build/_deps/googletest-src/googletest/src/gtest_main.cc
[==========] 36 tests from 3 test suites ran. (377 ms total)
[  PASSED  ] 36 tests.


####Debug build (device)

In [51]:
%cd $project_dir
!mkdir -p build
%cd build
!cmake --fresh -DTESTING=1 -DCMAKE_BUILD_TYPE=DebugDevice ..
!cmake --build . -j4
%cd tests

/root/cuda-sample1
/root/cuda-sample1/build
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Device debugging mode is enabled
-- Testing is enabled
Cloning into 'googletest-src'...
HEAD is now at b514bdc8 Update version strings to 1.15.2 (#4583)
-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Performing

In [52]:
!compute-sanitizer --target-processes all --tool memcheck --leak-check=full ./cuda-sample1-test --gtest_brief=1

========= COMPUTE-SANITIZER
Running main() from /root/cuda-sample1/build/_deps/googletest-src/googletest/src/gtest_main.cc
[==========] 36 tests from 3 test suites ran. (452 ms total)
[  PASSED  ] 36 tests.
========= LEAK SUMMARY: 0 bytes leaked in 0 allocations
========= ERROR SUMMARY: 0 errors


####Release build

In [53]:
%cd $project_dir
!mkdir -p build
%cd build
!cmake --fresh -DTESTING=1 -DBENCHMARKING=1 -DCMAKE_BUILD_TYPE=Release ..
!cmake --build . -j4
%cd tests

/root/cuda-sample1
/root/cuda-sample1/build
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Release mode is enabled
-- Testing is enabled
Cloning into 'googletest-src'...
HEAD is now at b514bdc8 Update version strings to 1.15.2 (#4583)
-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Performing Test CMA

In [54]:
!ctest

Test project /root/cuda-sample1/build/tests
      Start  1: AllocTest.floatScal
 1/36 Test  #1: AllocTest.floatScal ..............   Passed    0.13 sec
      Start  2: AllocTest.floatVecLen101
 2/36 Test  #2: AllocTest.floatVecLen101 .........   Passed    0.11 sec
      Start  3: AllocTest.intScal
 3/36 Test  #3: AllocTest.intScal ................   Passed    0.11 sec
      Start  4: AllocTest.intVecLen101
 4/36 Test  #4: AllocTest.intVecLen101 ...........   Passed    0.12 sec
      Start  5: AllocTest.unsignedScal
 5/36 Test  #5: AllocTest.unsignedScal ...........   Passed    0.12 sec
      Start  6: AllocTest.unsignedVecLen101
 6/36 Test  #6: AllocTest.unsignedVecLen101 ......   Passed    0.11 sec
      Start  7: TransTest.floatScal
 7/36 Test  #7: TransTest.floatScal ..............   Passed    0.11 sec
      Start  8: TransTest.floatVecLen101
 8/36 Test  #8: TransTest.floatVecLen101 .........   Passed    0.12 sec
      Start  9: TransTest.floatVecLen1001
 9/36 Test  #9: TransTest.fl

####Benchmarking

#####Linear job growth

In [55]:
#@markdown $G(x) = 100x + b$
%cd $project_dir/build/benchmarks

b = 400 # @param {type:"slider", min:100, max:1000, step:100}
x = 100 # @param {type:"slider", min:100, max:10000, step:100}

!./dotbench -b $b -n $x -o dot_host.csv
!./dotbench -b $b -n $x -o dot_device.csv -d

if x >= 200:
    %run vis.py -i dot_host.csv dot_device.csv -l Host Device -c indianred limegreen -t "Real Complexity of Dot Product (linear job)"
else:
    %run vis.py -i dot_host.csv dot_device.csv -l Host Device -c indianred limegreen -t "Real Complexity of Dot Product (linear job)" -m

/root/cuda-sample1/build/benchmarks
100/100[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ ] Host
100/100[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ ] Device


#####Exponential job growth

In [56]:
#@markdown $G(x) = 2^x + b$
%cd $project_dir/build/benchmarks

b = 1000 # @param {type:"slider", min:100, max:1000, step:100}
x = 19 # @param {type:"slider", min:1, max:25, step:1}

!./dotbench -e -b $b -n $x -o dot_host.csv
!./dotbench -e -b $b -n $x -o dot_device.csv -d

%run vis.py -i dot_host.csv dot_device.csv -l Host Device -c indianred limegreen -t "Real Complexity of Dot Product (exponential job)" -m

/root/cuda-sample1/build/benchmarks
19/19[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ ] Host
19/19[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ ] Device
